# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

## (1) Import the data and required Libraries

In [ ]:
import pandas as pd
import numpy as np
import math
import json
import datetime
% matplotlib inline



In [ ]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True, )
profile = pd.read_json('data/profile.json', orient='records', lines=True, )
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## (2) Explore the data

In [ ]:
portfolio.head()

In [ ]:
portfolio.shape

In [ ]:
profile.head()

In [ ]:
profile.shape

In [ ]:
transcript.head()

In [ ]:
transcript.shape

In [ ]:
portfolio.groupby('id').sum()

In [ ]:
persons = transcript.groupby('person').sum().sort_values(by='time',ascending=False).head()
persons

In [ ]:
transcript.dtypes

In [ ]:
portfolio.dtypes

In [ ]:
profile.dtypes

##### Profile dataframe exploration

In [ ]:
profile.groupby('age')['age',].count().plot(kind='bar',figsize=(12,9))

In [ ]:
profile.groupby('age').count().plot(kind='bar',figsize=(25,9))

In [ ]:
profile[profile.age == 118].count()

In [ ]:
profile.count()

In [ ]:
profile.isnull().sum()

In [ ]:
profile.shape

In [ ]:
profile.groupby('age')['age','income'].count().plot(kind='bar',figsize=(20,9))

In [ ]:
profile.groupby('gender')['gender'].count().plot(kind='bar',figsize=(20,9))

In [ ]:
profile.groupby('age')['age'].count().plot(kind='bar',figsize=(20,9))

##### exploring transcript data frame

In [ ]:
transcript.groupby('event')['event'].count().plot(kind='bar')

In [ ]:
transcript.time.plot(kind='hist')

In [ ]:
transcript.time.describe()

In [ ]:
transcript.value.head(10)

In [ ]:
transcript.groupby('person')['person'].count()

In [ ]:
transcript.head()

## Cleaning

##### Profile dataframe cleaning

In [ ]:
clean_profile = profile.copy()

In [ ]:
clean_profile.dropna(axis=0, inplace=True)

In [ ]:
clean_profile = clean_profile.rename(columns={'id':'customer_id'})

In [ ]:
clean_profile = clean_profile.reindex(columns=['customer_id', 'age', 'became_member_on', 'gender', 'income'])

In [ ]:
# replacing the 'customer_id' string values  with easiest numerical values
def map_customer_id(df,col):
    labels_customer_id = df[col].astype('category').cat.categories.tolist()
    map_customer_id = {col : {k: v for k,v in zip(labels_customer_id,list(range(1,len(labels_customer_id)+1)))}}
    df.replace(map_customer_id, inplace=True)
    return df

In [ ]:
clean_profile = map_customer_id(clean_profile,'customer_id')

In [ ]:
clean_profile.became_member_on = pd.to_datetime(clean_profile.became_member_on, format='%Y%m%d', errors='ignore')

In [ ]:
clean_profile['membership_year'] = clean_profile.became_member_on.dt.year
clean_profile.membership_year.head()

In [ ]:
clean_profile['membership_month'] = clean_profile.became_member_on.dt.month
clean_profile.membership_month.head()

In [ ]:
clean_profile['membership_days'] = datetime.datetime.today().date() - clean_profile['became_member_on'].dt.date
clean_profile.membership_days.head()
clean_profile['membership_days'] = clean_profile['membership_days'].dt.days

In [ ]:
# replacing the 'customer_id' string values  with easiest numerical values
labels_gender = clean_profile['gender'].astype('category').cat.categories.tolist()
labels_gender

In [ ]:
map_gender = {'gender' : {k: v for k,v in zip(labels_gender,list(range(1,len(labels_gender)+1)))}}

In [ ]:
clean_profile.replace(map_gender, inplace=True)

In [ ]:
clean_profile.head()

In [ ]:
clean_profile['membership_year'].dtype

In [ ]:
clean_profile['membership_days'].plot(kind='hist', title = 'membership duration frequency', x='membership_days')

In [ ]:
clean_profile['membership_days'].describe()

In [ ]:
clean_profile.groupby(clean_profile.became_member_on.dt.year)['customer_id'].count().plot(kind='bar')

##### Transcipt dataframe cleaning

In [ ]:
clean_transcript = transcript.copy()

In [ ]:
# renaming 'person' column to 'customer_id'

clean_transcript = clean_transcript.rename(columns={'person':'customer_id'})

In [ ]:
clean_transcript = clean_transcript.reindex(columns=['customer_id','event','time','value'])

In [ ]:
# replace categorical labels in 'customer_id' column with numerical labels

clean_transcript = map_customer_id(clean_transcript,'customer_id')

In [ ]:
clean_transcript.head()

In [ ]:
clean_transcript.dtypes

In [ ]:
clean_transcript.value.head()

In [ ]:
# getting the different keys  that exists in the 'value' column
keys = []
for idx, row in clean_transcript.iterrows():
    for k in row['value']:
        if k in keys:
            continue
        else:
            keys.append(k)

In [ ]:
# checking the different keys of the 'value' dictionary
keys

In [ ]:
#create columns and specify the datatype of each of them
clean_transcript['offer_id'] = '' # datatype : string
clean_transcript['amount'] = 0  # datatype : integer
clean_transcript['reward'] = 0  # datatype : integer

In [ ]:
# then updating it and using the values to fill in the columns created above
for idx, row in clean_transcript.iterrows():
    for k in row['value']:
        if k == 'offer_id' or k == 'offer id': # b/c 'offer_id' and 'offer id' are representing the same thing 
            clean_transcript.at[idx, 'offer_id'] = row['value'][k]
        if k == 'amount':
            clean_transcript.at[idx, 'amount'] = row['value'][k]
        if k == 'reward':
            clean_transcript.at[idx, 'reward'] = row['value'][k]

In [ ]:
# filling all the NaNs in the 'offer_id' column with 'N/A' values (i.e. Not Applicable)
clean_transcript['offer_id'] = clean_transcript['offer_id'].apply(lambda x: 'N/A' if x == '' else x)

In [ ]:
#check the created columns and their values
clean_transcript.tail()

In [ ]:
# dropping the 'value' column 
clean_transcript.drop('value', axis=1, inplace=True)

In [ ]:
#check the created columns and their values
clean_transcript.head()

###### event column preprocessing
Two important points to keep in mind:
1. Since we are interested in the events related with the offers, i.e offer received, offer viewed, offer completed, we will remove all events of 'transaction' because they are not directly related with offers.
2. we will exclude all the events of 'offer recieved',since I want to focus on whether the customer:
Only viewed the offer (offer viewed)
viewed the offer and then completed it (offer viewed)&(offer completed).

In [ ]:
# excluding all events of 'transaction' from our clean_transcript dataset
clean_transcript = clean_transcript[clean_transcript['event'] != 'transaction']

# excluding all events of 'offer received' 
clean_transcript = clean_transcript[clean_transcript['event'] != 'offer received']

In [ ]:
# checking that the events of either 'transaction' or 'offer received'were successfully removed from the dataset
clean_transcript['event'].unique()

In [ ]:
# replacing the 'event' categorical labels with coressponding numerical label
labels_event = clean_transcript['event'].astype('category').cat.categories.tolist()
replace_map_comp_event = {'event' : {k: v for k,v in zip(labels_event,list(range(1,len(labels_event)+1)))}}

In [ ]:
# checking the numerical label and its corresponding categorical label
print(replace_map_comp_event)

In [ ]:
# replace categorical labels in 'event' column with numerical labels
clean_transcript.replace(replace_map_comp_event, inplace=True)

In [ ]:
clean_transcript.head()

Preprocessing 'offer_id' Feature

In [ ]:
# replacing the 'event' categorical labels with coressponding numerical label
labels_offer_id = clean_transcript['offer_id'].astype('category').cat.categories.tolist()
replace_map_comp_offer_id = {'offer_id' : {k: v for k,v in zip(labels_offer_id,list(range(1,len(labels_offer_id)+1)))}}

In [ ]:
print(replace_map_comp_offer_id)

In [ ]:
# replacing the categorical values in the 'offer_id' column by its corresponding numerical values
clean_transcript.replace(replace_map_comp_offer_id, inplace=True)

In [ ]:
clean_transcript.head()

##### cleaning portfolio dataframe

In [ ]:
clean_portfolio = portfolio.copy()


In [ ]:
clean_portfolio= clean_portfolio.rename(columns={'id':'customer_id'})

In [ ]:
clean_portfolio = clean_portfolio.reindex(columns=['customer_id','offer_type','dificulty','reward','duration','channels'])

In [ ]:
# replacing the 'customer_id' string values  with easiest numerical values
clean_portfolio = map_customer_id(clean_portfolio,'customer_id')

In [ ]:
clean_portfolio.tail()